In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from numpy import cos
from math import log
from scipy.interpolate import interp1d
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize

#### As equações do modelo:

$dShdt = \mu_H N -a  k b_2  (I_M / N)  S_H - \mu_H S_H\\
dIhdt = a  k b_2  (I_M / N)  S_H - \gamma  I_H - \mu_H I_H\\
dRhdt = \gamma  I_H - \mu_H R_H$

$dSmdt = b - a  k b_1  (I_H / N)  S_M - \mu S_M \\
dEmdt = a k b_1  (I_H / N)  S_M - \mu E_M - b_3 E_M - l E_M\\
dImdt = b_3  E_M  - \mu I_M$

In [2]:
%display typeset

In [3]:
var('S_H I_H R_H S_M E_M I_M b a b1 b2 k gamma N mu b3 l mu_H')

(S_H, I_H, R_H, S_M, E_M, I_M, b, a, b1, b2, k, gamma, N, mu, b3, l, mu_H)

In [4]:
dShdt = mu_H * N -a * k * b2 * (I_M / N) * S_H - mu_H * S_H
dIhdt = a * k * b2 * (I_M / N) * S_H - gamma * I_H - mu_H * I_H
dRhdt = gamma * I_H - mu_H * R_H
dSmdt = b - a * k * b1 * (I_H / N) * S_M - mu * S_M
dEmdt = a * k * b1 * (I_H / N) * S_M - mu * E_M - b3 * E_M - l * E_M
dImdt = b3 * E_M - mu * I_M

#### Calculando o equilíbrio do SIR/SEI

In [5]:
solve([dShdt, dIhdt, dRhdt, dSmdt, dEmdt, dImdt],[S_H, I_H, R_H, S_M, E_M, I_M])

[[S_H == N, I_H == 0, R_H == 0, S_M == b/mu, E_M == 0, I_M == 0], [S_H == (N^2*gamma*mu^3 + (N^2*b3*gamma + N^2*gamma*l)*mu^2 + (N^2*mu^3 + (N^2*a*b1*k + N^2*b3 + N^2*l)*mu^2 + (N^2*a*b1*b3*k + N^2*a*b1*k*l)*mu)*mu_H)/(a^2*b*b1*b2*b3*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H), I_H == -((N^2*mu^3 + (N^2*b3 + N^2*l)*mu^2)*mu_H^2 - (N*a^2*b*b1*b2*b3*k^2 - N^2*gamma*mu^3 - (N^2*b3*gamma + N^2*gamma*l)*mu^2)*mu_H)/(a^2*b*b1*b2*b3*gamma*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H^2 + (a^2*b*b1*b2*b3*k^2 + N*a*b1*gamma*k*mu^2 + (N*a*b1*b3*gamma*k + N*a*b1*gamma*k*l)*mu)*mu_H), R_H == (N*a^2*b*b1*b2*b3*gamma*k^2 - N^2*gamma^2*mu^3 - (N^2*b3*gamma^2 + N^2*gamma^2*l)*mu^2 - (N^2*gamma*mu^3 + (N^2*b3*gamma + N^2*gamma*l)*mu^2)*mu_H)/(a^2*b*b1*b2*b3*gamma*k^2 + (N*a*b1*k*mu^2 + (N*a*b1*b3*k + N*a*b1*k*l)*mu)*mu_H^2 + (a^2*b*b1*b2*b3*k^2 + N*a*b1*gamma*k*mu^2 + (N*a*b1*b3*gamma*k + N*a*b1*gamma*k*l)*mu)*mu_H), S_M == (a*b*b2*b3*gamma*k + (N*mu^2 + (N*b3 + N*l)*mu)*mu_H^2 + (a*b*b2*b3*k + N*gamma*mu^2 + (N*b3*gamma + N*gamma*l)*mu)*mu_H)/(a*b2*b3*gamma*k*mu + (a^2*b1*b2*b3*k^2 + a*b2*b3*k*mu)*mu_H), E_M == -((N*mu^3 + (N*b3 + N*l)*mu^2)*mu_H^2 - (a^2*b*b1*b2*b3*k^2 - N*gamma*mu^3 - (N*b3*gamma + N*gamma*l)*mu^2)*mu_H)/(a*b2*b3*gamma*k*mu^2 + (a*b2*b3^2*gamma*k + a*b2*b3*gamma*k*l)*mu + (a^2*b1*b2*b3^2*k^2 + a^2*b1*b2*b3*k^2*l + a*b2*b3*k*mu^2 + (a^2*b1*b2*b3*k^2 + a*b2*b3^2*k + a*b2*b3*k*l)*mu)*mu_H), I_M == -((N*mu^3 + (N*b3 + N*l)*mu^2)*mu_H^2 - (a^2*b*b1*b2*b3*k^2 - N*gamma*mu^3 - (N*b3*gamma + N*gamma*l)*mu^2)*mu_H)/(a*b2*gamma*k*mu^3 + (a*b2*b3*gamma*k + a*b2*gamma*k*l)*mu^2 + (a*b2*k*mu^3 + (a^2*b1*b2*k^2 + a*b2*b3*k + a*b2*k*l)*mu^2 + (a^2*b1*b2*b3*k^2 + a^2*b1*b2*k^2*l)*mu)*mu_H)]]

Calculando com parâmetros fixos, vamos calcular o equilíbrio no tempo final:

In [6]:
T_linha = 25.6
T1= 26.4 
T2= 0.025
omega1= 0.017
phi1= -1.45

R1= 250.083 
R2= 0.565
omega2= 0.02
phi2=1.6

B_E=200
p_ME=0.9
p_ML=0.75
p_MP=0.85
tau_E=1
tau_P=1
# c1=0.019
# c2=-0.021
c1=0.00554
c2=-0.06737
mu_H = 0.00007

D1=36.5
b1=0.04

A=12.5
B=15
C=-48.78

b2=0.09
tau_H=10
DD=105
Tmin=14.5
gamma= 1/120 #365 
R_L = 450
N = 7717 #População estimada para 2004
M = 300000 
k=1 #Análise sem efeito do desmatamento

E_M0 = 1 #Analisando a infecção iniciando com apenas um mosquito exposto e um humano infectado
I_H0 = 1
I_M0 = 0
R_H0 = 0
if E_M0 is None:
        E_M0 = M / 100

S_H0 = N - I_H0 - R_H0
S_M0 = M - E_M0 - I_M0

In [7]:
def Temp(t):
    return T1 * (1 + T2 * cos(omega1 * t - phi1))

def Rain(t):
    return R1 * (1 + R2 * cos(omega2 * t - phi2))

R_final = Rain(1825)
T_final = Temp(1825)
    
# Duração da fase de desenvolvimento das larvas
def tau_L(Temp):
    return 1 / (c1 * Temp + c2)
    
# Taxa diária de sobrevivência dos mosquitos
def p(Temp):
    return np.exp(-1 / (A*Temp**2 + B*Temp + C))
    
# Probabilidade de sobrevivência das larvas dependente de temperatura e chuva
def p_LT(Temp):
    return np.exp(-(c1 * Temp + c2))
    
def p_LR(Rain):
    return (4*p_ML/R_L**2) * Rain * (max(10^-5,R_L - Rain))
    
def p_LRT(Rain,Temp):
    return p_LR(Rain)*p_LT(Temp)
    
# Probabilidade de sobrevivência dos ovos e pupas dependente de chuva (Não definido previamente)
def p_ER(Rain):
    return (4*p_ME/R_L**2) * Rain * (max(10^-5,R_L - Rain))
    
# Probabilidade de sobrevivência dos ovos e pupas dependente de chuva (Não definido previamente)
def p_PR(Rain):
    return (4*p_MP/R_L**2) * Rain * (max(10^-5,R_L - Rain))

# Duração do ciclo esporogônico
def tau_M(Temp):
    return DD / (max(10^-5,Temp - Tmin))
    
tau_M_final = tau_M(T_final)

# Taxa de infecção de expostos (1/Duração do ciclo esporogônico)
def b3(tau_M, Temp):
    return 1/tau_M
    
b3_final = b3(tau_M_final, T_final)
    
# Definindo a função de sobrevivência do mosquito
def l_tau_M(tau_M, Temp):  
    p_T = p(Temp)
    return p_T**tau_M
    
l_final = l_tau_M(tau_M_final, T_final)

# Definindo a taxa de mortalidade do mosquito
def mu(Temp):
    return -log(p(Temp))
    
mu_final = mu(T_final)
    
# Calcula a taxa de picada e probabilidade de transmissão
def a(Temp):
    return (max(10^-5, Temp - T_linha)) / D1
    
a_final = a(T_final)
    
def b(Rain,Temp):
    return B_E * p_ER(Rain) * p_LRT(Rain,Temp) * p_PR(Rain) / (tau_E + tau_L(Temp) + tau_P)
    
b_final = b(R_final, T_final)

In [8]:
def R0_sir(a, k, b2, gamma, mu_H):
    return abs((a * k * b2) / (gamma + mu_H))
    
#R0_sir_curr = R0_sir(a_curr, b2, gamma)
    
def R0_sei(a, k, b1, b3, l, mu):
    return abs((a * k * b1 * b3) / ((b3 + l) * mu + mu** 2))
    
#R0_sei_curr = R0_sei(a_curr, b1, b3_curr, l_curr, mu_curr)
    
def R0_full(a, k, b1, b2, b3, gamma, l, mu, mu_H):
    return abs(((a**2 * b1 * b2 * b3 * k**2) / (mu*(gamma + mu_H)*(b3 + l + mu)))**(1/2))
    
#R0_curr = R0(a_curr, b1, b2, b3_curr, gamma, l_curr, mu_curr)

def calculate_initial_values(R_L, T_linha, Tmin, DD, D1, T1, T2, omega1, phi1, R1, R2, omega2, phi2,
                            p_ML, p_ME, p_MP, c1, c2, A, B, C, b1, b2, gamma, mu_H, k):
    
    T0 = T1 * (1 + T2 * cos(omega1 * 0 - phi1))
    R0 = R1 * (1 + R2 * cos(omega2 * 0 - phi2))

    tau_L0 = 1 / (c1 * T0 + c2)

    p_LT0 = np.exp(-(c1 * T0 + c2))
    
    p_LR0 = (4 * p_ML / R_L**2) * R0 * (max(1e-5, R_L - R0))
    
    p_LRT0 = p_LR0 * p_LT0
    
    p_ER0 = (4 * p_ME / R_L**2) * R0 * (max(1e-5, R_L - R0))
    
    p_PR0 = (4 * p_MP / R_L**2) * R0 * (max(1e-5, R_L - R0))

    a0 = (max(1e-5, T0 - T_linha)) / D1
    b0 = B_E * p_ER0 * p_LRT0 * p_PR0 / (tau_E + tau_L0 + tau_P)

    tau_M0 = DD / (max(1e-5, T0 - Tmin))
    
    b30 =  1 / tau_M0
    
    p0 = np.exp(-1 / (A * T0**2 + B * T0 + C))
    l0 = p0 ** tau_M0

    mu0 = -log(p0)

    initial_R0_sir = R0_sir(a0, k, b2, gamma, mu_H)
    initial_R0_sei = R0_sei(a0, k, b1, b30, l0, mu0)
    initial_R0_full = R0_full(a0, k, b1, b2, b30, gamma, l0, mu0, mu_H)
    
    return initial_R0_sir, initial_R0_sei, initial_R0_full, a0

In [9]:
initial_R0_sir, initial_R0_sei, initial_R0_full, a0 = calculate_initial_values(R_L, T_linha, Tmin, DD, D1, 
                                                                               T1, T2, omega1, phi1, R1, R2, omega2, phi2,
                                                                               p_ML, p_ME, p_MP, c1, c2, A, B, C, b1, b2, 
                                                                               gamma, mu_H, k)

In [10]:
print(f'R0_sir = {initial_R0_sir} \nR0_sei = {initial_R0_sei}\nR0_acoplado = {initial_R0_full}\nTaxa de picadas = {a0}')

R0_sir = 0.258077185614824 
R0_sei = 0.9002047882569278
R0_acoplado = 0.4819982554225031
Taxa de picadas = 0.02409676240499894


In [11]:
initial_R0_sir, initial_R0_sei, initial_R0_full, a0 = calculate_initial_values(R_L, T_linha, Tmin, DD, D1, 
                                                                               T1, T2, omega1, phi1, R1, R2, omega2, phi2,
                                                                               p_ML, p_ME, p_MP, c1, c2, A, B, C, b1, b2, 
                                                                               gamma, mu_H, 10)

In [12]:
print(f'R0_sir = {initial_R0_sir} \nR0_sei = {initial_R0_sei}\nR0_acoplado = {initial_R0_full}\nTaxa de picadas = {a0}')

R0_sir = 2.5807718561482402 
R0_sei = 9.00204788256928
R0_acoplado = 4.819982554225031
Taxa de picadas = 0.02409676240499894
